In [1]:
import pickle
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tqdm import tqdm
import numpy as np
import os

ModuleNotFoundError: No module named 'tensorflow'

In [23]:
with open("NetworkTransition.txt0_90.txt") as nt:
    mainData = nt.readlines()
mainData = [i.strip(" \n").split(" ") for i in mainData]
mainData

[['0', '1', '0', '0', '1', '1', '0', '1', '0', '0'],
 ['0', '1', '0', '0', '0', '1', '0', '0', '0', '0'],
 ['0', '1', '0', '0', '0', '1', '1', '0', '1', '0'],
 ['0', '1', '1', '0', '0', '1', '1', '1', '1', '0'],
 ['1', '1', '1', '0', '1', '1', '1', '1', '0', '0'],
 ['1', '1', '0', '0', '1', '1', '0', '1', '0', '0'],
 ['0', '0', '0', '0', '1', '1', '0', '0', '0', '0'],
 ['0', '1', '0', '1', '0', '0', '0', '0', '1', '1'],
 ['0', '1', '0', '0', '1', '1', '1', '1', '1', '0'],
 ['0', '1', '1', '0', '0', '1', '0', '1', '0', '0'],
 ['1', '1', '0', '0', '1', '1', '1', '0', '0', '0'],
 ['0', '1', '0', '0', '1', '1', '0', '1', '1', '0'],
 ['0', '1', '1', '0', '0', '1', '0', '0', '0', '0'],
 ['1', '1', '0', '0', '1', '1', '1', '0', '1', '0'],
 ['0', '1', '1', '0', '1', '1', '0', '1', '1', '0'],
 ['1', '1', '1', '0', '1', '1', '0', '0', '0', '0'],
 ['1', '1', '0', '0', '1', '1', '0', '0', '1', '0'],
 ['0', '0', '1', '0', '1', '1', '0', '0', '1', '0'],
 ['1', '1', '1', '1', '1', '1', '0', '0', '1',

In [24]:
# with open('models_dict.pkl', 'rb') as file:
#     loaded_models_dict = pickle.load(file)

In [25]:
import os
from tensorflow.keras.models import load_model

# Directory where models are saved
save_dir = 'M/models6'

# Initialize the dictionary to store models
loaded_models_dict = {}

# Loop through all files in the directory
for file_name in os.listdir(save_dir):
    if file_name.endswith('.keras'):
        # Extract key and sub_key from the file name
        # Assuming file names are in the format "model_key_sub_key.keras"
        parts = file_name.split('_')
        key = int(parts[1])  # Convert to integer, assuming keys are integers
        sub_key = int(parts[2].split('.')[0])  # Extract sub_key and remove '.keras'

        # Load the model
        model_path = os.path.join(save_dir, file_name)
        model = load_model(model_path)

        # Store the model in the dictionary
        if key not in loaded_models_dict:
            loaded_models_dict[key] = {}
        loaded_models_dict[key][sub_key] = model


In [26]:
def generate_combinations_with_indices(arr, r):
    n = len(arr)
    indices = list(range(r))
    result = []

    while True:
        # Create a dictionary with index-value pairs
        combination_dict = {indices[i]: arr[indices[i]] for i in range(r)}
        result.append(combination_dict)

        # Find the rightmost index that can be incremented
        for i in reversed(range(r)):
            if indices[i] != i + n - r:
                break
        else:
            return result  # No more combinations can be generated

        # Increment this index
        indices[i] += 1

        # Update the indices that follow
        for j in range(i + 1, r):
            indices[j] = indices[j - 1] + 1

def generate_permutations_with_indices(arr, r):
    from itertools import permutations
    n = len(arr)
    result = []

    # Generate all possible permutations using indices
    for perm in permutations(range(n), r):
        # Create a dictionary with index-value pairs
        permutation_dict = {i: arr[i] for i in perm}
        result.append(permutation_dict)

    return result


In [27]:
def remove_elements_from_row(data, row, indexListToRemove):
    selected_row = data[row]
    result = [val for idx, val in enumerate(selected_row) if idx not in indexListToRemove]

    return result

In [28]:
from boolF import getBoolF
print(getBoolF("NetworkTransition.txt0_90.txt"))

{3: {'targetGenes': (1,), 'mismatch': 0}, 6: {'targetGenes': (4,), 'mismatch': 0}, 8: {'targetGenes': (7,), 'mismatch': 0}, 9: {'targetGenes': (1,), 'mismatch': 0}, 0: {'targetGenes': (2,), 'mismatch': 0}}


In [29]:
from boolF import getBoolF
size = 1
ff = []

print("********* BOOLF RES ***********")
temp1 = getBoolF("NetworkTransition.txt0_90.txt")
print(temp1)
print("********* BOOLF RES ***********")
print("********* NN INFERENCE ***********")
timeStamp = 20
numOfGenes = 10 
tgI = 0
finalResult = getBoolF("NetworkTransition.txt0_90.txt")

while tgI != numOfGenes:
    minMismatch = 1000000
    datasetDict = {}
    print("*"*50)
    print(f"Target gene Index: {tgI}")
    temp1 = getBoolF("NetworkTransition.txt0_90.txt")
    indexListToRemove = []
    for k, v in temp1.items():
        indexListToRemove.append(k)
    
    if tgI in indexListToRemove:
        tgI+=1
        continue
    else:
        indexListToRemove.append(tgI)
    
    print(indexListToRemove)
    
    yList = [i[tgI] for i in mainData]
    for t0 in range(timeStamp):
        if t0+1 == timeStamp or tgI == numOfGenes:
            break
        # x = mainData[t0][:tgI] + mainData[t0][tgI+1:]
        x = mainData[t0]
        # x = remove_elements_from_row(mainData, t0, indexListToRemove)
        alreadyTaken = {}

        x = [float(v) for v in x]
        for r in range(1, len(x) + 1):
            comb = generate_combinations_with_indices(x, r)
            for cc in comb:
                keyList = []
                valList = []
                # print(i)
                for k, v in cc.items():
                    if k in indexListToRemove:
                        continue
                    keyList.append(k)
                    valList.append(v)
                if len(keyList) == 0:
                    continue
                # print(f"KEYS: {keyList}   VALS: {valList}")
                if tuple(keyList) not in alreadyTaken:
                    alreadyTaken[tuple(keyList)] = 1
                    if tuple(keyList) not in datasetDict:
                        datasetDict[tuple(keyList)] = []

                    datasetDict[tuple(keyList)].append(valList)
        
    
    print(len(datasetDict))
    for k, v in tqdm(datasetDict.items(), total=len(datasetDict)):
        x = v
        predList = []
        probPred = []
        
        # print(k, v)
        for p in x:
            predData = np.array([p])
            predictions = loaded_models_dict[tgI][len(p)].predict(predData, verbose=0)
            # threshold = 0.13
            # prediction = [1.0 if pred > threshold else 0.0 for pred in predictions]
            # predList.append(prediction[0])
            probPred.append(list(predictions)[0][0])

        max_value = max(probPred) - 0.1

        predList = [1 if value >= max_value else 0 for value in probPred]
        yy = yList[1:]
        yy = [float(b) for b in yy]
        mismatches = sum([1 for pred, actual in zip(predList, yy) if pred != actual])

        if mismatches < minMismatch:
            print(f"Previous minMismatch: {minMismatch}   Improved: {mismatches}")
            minMismatch = mismatches
            finalResult[tgI] = {"targetGenes": k, "mismatch": mismatches}
            if mismatches == 0:
                break

        # finalModels[tgI] = allModels
    tgI+=1
    print(finalResult)
    # break

********* BOOLF RES ***********
{3: {'targetGenes': (1,), 'mismatch': 0}, 6: {'targetGenes': (4,), 'mismatch': 0}, 8: {'targetGenes': (7,), 'mismatch': 0}, 9: {'targetGenes': (1,), 'mismatch': 0}, 0: {'targetGenes': (2,), 'mismatch': 0}}
********* BOOLF RES ***********
********* NN INFERENCE ***********
**************************************************
Target gene Index: 0
**************************************************
Target gene Index: 1
[3, 6, 8, 9, 0, 1]
15


  7%|▋         | 1/15 [00:00<00:11,  1.19it/s]

Previous minMismatch: 1000000   Improved: 9


 13%|█▎        | 2/15 [00:01<00:10,  1.21it/s]

Previous minMismatch: 9   Improved: 8


...

Previous minMismatch: 8   Improved: 7


 73%|███████▎  | 11/15 [00:09<00:03,  1.20it/s]

Previous minMismatch: 7   Improved: 6


100%|██████████| 15/15 [00:12<00:00,  1.20it/s]

{3: {'targetGenes': (1,), 'mismatch': 0}, 6: {'targetGenes': (4,), 'mismatch': 0}, 8: {'targetGenes': (7,), 'mismatch': 0}, 9: {'targetGenes': (1,), 'mismatch': 0}, 0: {'targetGenes': (2,), 'mismatch': 0}, 1: {'targetGenes': (5,), 'mismatch': 3}, 2: {'targetGenes': (1,), 'mismatch': 11}, 4: {'targetGenes': (2,), 'mismatch': 5}, 5: {'targetGenes': (1, 2, 4), 'mismatch': 0}, 7: {'targetGenes': (1, 2, 4), 'mismatch': 6}}
**************************************************
Target gene Index: 8
**************************************************
Target gene Index: 9


In [30]:
finalResult

{3: {'targetGenes': (1,), 'mismatch': 0},
 6: {'targetGenes': (4,), 'mismatch': 0},
 8: {'targetGenes': (7,), 'mismatch': 0},
 9: {'targetGenes': (1,), 'mismatch': 0},
 0: {'targetGenes': (2,), 'mismatch': 0},
 1: {'targetGenes': (5,), 'mismatch': 3},
 2: {'targetGenes': (1,), 'mismatch': 11},
 4: {'targetGenes': (2,), 'mismatch': 5},
 5: {'targetGenes': (1, 2, 4), 'mismatch': 0},
 7: {'targetGenes': (1, 2, 4), 'mismatch': 6}}

In [18]:
loaded_models_dict

{0: {1: <Sequential name=sequential, built=True>,
  2: <Sequential name=sequential_1, built=True>,
  3: <Sequential name=sequential_2, built=True>,
  4: <Sequential name=sequential_3, built=True>,
  5: <Sequential name=sequential_4, built=True>},
 1: {1: <Sequential name=sequential_5, built=True>,
  2: <Sequential name=sequential_6, built=True>,
  3: <Sequential name=sequential_7, built=True>,
  4: <Sequential name=sequential_8, built=True>,
  5: <Sequential name=sequential_9, built=True>},
 2: {1: <Sequential name=sequential_10, built=True>,
  2: <Sequential name=sequential_11, built=True>,
  3: <Sequential name=sequential_12, built=True>,
  4: <Sequential name=sequential_13, built=True>,
  5: <Sequential name=sequential_14, built=True>},
 3: {1: <Sequential name=sequential_15, built=True>,
  2: <Sequential name=sequential_16, built=True>,
  3: <Sequential name=sequential_17, built=True>,
  4: <Sequential name=sequential_18, built=True>,
  5: <Sequential name=sequential_19, built=True

In [2]:
def read_actual_data(file_path, value_dict):
    actual_data = {}
    with open(file_path, 'r') as file:
        for line in file:
            parts = line.split()  # Split the line into words
            if not parts:
                continue  # Skip empty lines
            key = value_dict[parts[0]]  # Map the first item (key) using the dictionary
            actual_data[key] = set(value_dict[part] for part in parts[1:] if part in value_dict)  # Map the rest of the line
    return actual_data

In [5]:

cell_cycle_dict = {
    "Cln3": 0,
    "MBF": 1,
    "SBF": 2,
    "Cln1": 3,
    "Cdh1": 4,
    "Swi5": 5,
    "Cdc20": 6,
    "Clb5": 7,
    "Sic1": 8,
    "Clb1": 9,
    "Mcm1": 10
}
actual_data = read_actual_data("OrigData/new.txt", cell_cycle_dict)
actual_data

{0: set(),
 1: {0, 9},
 2: {0, 9},
 3: {2},
 4: {3, 6, 7, 9},
 5: {6, 9, 10},
 6: {9, 10},
 7: {1, 6, 8},
 8: {3, 5, 6, 7, 9},
 9: {4, 6, 7, 8, 10},
 10: {7, 9}}

In [6]:
def read_actual_data(file_path):
    actual_data = {}
    with open(file_path, 'r') as file:
        for line in file:
            parts = list(map(int, line.split()))
            key = parts[0]
            actual_data[key] = set(parts[1:])
    return actual_data

In [8]:
actual_data = read_actual_data("New Text Document.txt")
actual_data

{0: {1, 3},
 1: {2, 7, 8},
 2: {5, 6},
 3: {0, 6, 7},
 4: {0},
 5: {2, 6},
 6: {2},
 7: {0, 2, 5, 8},
 8: {4},
 9: {5}}